In [ ]:
This pipeline halp Sagemaker connect to Database, query files and upload target files to S3 bucket.

In [1]:
pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.


In [2]:
import mysql.connector

In [3]:
mydb = mysql.connector.connect(
  host="...",
  user="admin",
  password="...",
  database="..."
)

In [4]:
print(mydb)
mycursor = mydb.cursor()

In [5]:
mycursor.execute("Select summary_file_key from summary_file_records where condition_id in (select condition_id from v_base_data where campaign_id = 10151)")

In [6]:
myresult = mycursor.fetchall()

In [7]:
ans = []
for x in myresult:
    str_x = ''.join(x)
    ans.append(str_x)

In [ ]:
print(ans)

In [12]:
import pandas as pd
df = pd.DataFrame(data=ans, columns=['file'])

In [ ]:
df

In [16]:
import boto3
s3 = boto3.client('s3', region_name='us-east-2')

In [28]:
for item in df['file']:
    _, filename = item.split('/',maxsplit=1)
    bucket = 'hits-prod'
    target_parent = 'download_files'
    s3.download_file(bucket, item, 'temp.csv')
    s3.upload_file('temp.csv', bucket, target_parent+'/'+filename)

In [19]:
df.iloc[0]['file']

'summary_files_internal/306228_10061_10151_1419_3701_10206_summary.csv'

In [21]:
s3.download_file('hits-prod','summary_files_internal/306228_10061_10151_1419_3701_10206_summary.csv', 'temp.csv')

In [22]:
s3.upload_file('temp.csv', 'hits-prod','download_files/306228_10061_10151_1419_3701_10206_summary.csv')